<a href="https://colab.research.google.com/github/FaresAmiar/inference_sampling/blob/main/Decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Algorithme du reject sampling

In [ ]:
import random
from datetime import datetime 
import time


class Network:
  
  def __init__(self,name, vars = None):
    self.name = name
    if vars == None:
      vars = []
    self.variables = vars

  def addVariable(self, var):
    self.variables.append(var)

  def getVariables(self):
    return self.variables

  def addRequest(self, request):
    self.request = request
  
  def getRequest(self):
    return self.request

  def toString(self):
    str = ""
    for var in self.variables:
      str += var.toString() + "\n"

  #renvoie la requête entière sous forme de tuple,
  #avec en premier la variable requêtée, ensuite les valeurs demandées (ou non)
  #pour chaque variable, enfin, les valeurs d'échantillonage demandées
  def normalizeRequest(self):
    nex = []
    request = self.request
    found = False
    first = request[request.find("(")+1:request.find("|")].strip()
    others = request[request.find("|")+1 : request.find(")")].strip().split(",")

    
    i = 0
    nex.append(first)

    
    for var in self.getVariables():
      #on regarde si le nom de la variable est trouvée dans la requête
      print("i",i)
      i += 1
      for s in filter (lambda x: var.name in x, others): 
        boo = s[s.find("=")+1:]
        if boo == "0":
          boo = "False"
        elif boo == "1":
          boo = "True"
        boo = boo[0].upper() + boo[1:]
        nex.append(boo)
        found = True
        print("found",found)
      if(not found):
        boo = ""
        print("boo",boo)
        nex.append(boo)
      
      found = False

    
    nex.append(request[request.rfind("[")+1:request.rfind(",")])
    nex.append(request[request.rfind(".")-1: request.find("]")])

    return tuple(nex)

  #Transforme le résultat précédent au format des dépencences de la variable
  def adaptResultToDependencies(self, var ,result):
    varIdxs = {}
    for i in range(len(self.getVariables())):
      varIdxs[self.getVariables()[i].name] = i
    rIdx = []

    for d in var.getDependencies():
     rIdx.append(varIdxs[d])
    
    newT = []

    for idx in rIdx:
      newT.append(result[idx])
    
    return newT


  #algorithme du reject sampling
  def rejectSampling(self):
    
    p = None
    res = []

    req = self.normalizeRequest()
    first = req[0]
    req = req[1:]
    i = 0
    broken = False #si une des conditions de la requete est rompue
    algoIterations = 0

    rates = [0,0]
    nbTrueFalse = [0,0]

    pSelecteds = {}

    
    print("req",req)

    #tant que l'on n'a pas un écart epsilon entre l'échantillonage calculé et celui de la requete:
    
    epsilon = abs(rates[0] - float(req[-2]))
    bestEpsilon = float(200000000)
    bestSample = []
    
    while(epsilon != 0 and algoIterations <= 1000000):

      for v in self.getVariables():
        #les variables sans dépendences
        if(len(v.getDependencies()) == 0):
          p = v.getProbas()[0]
          booRes, pSelected = self.getBoolAndProbaFromSample(p)
          res.append(booRes)
          pSelecteds[v.name] = pSelected 
        else:
          tmpRes = self.adaptResultToDependencies(v,res)

          p = v.probaFromDependenciesVal(tmpRes)

          booRes, pSelected = self.getBoolAndProbaFromSample([p[0][-2],p[0][-1]])
          pSelecteds[v.name] = pSelected

          #si une valeur est demandée et que la valeur trouvée est différente
          if req[i] != "" and booRes != req[i]:
            broken = True #on rejette
            break
          
          res.append(booRes)
        i += 1
        
        
      #print(pSelecteds)

      
      #si on a un résultat:
      if not broken :
        varIdxs = {}
        for i in range(len(self.getVariables())):
          varIdxs[self.getVariables()[i].name] = i
        rIdx = varIdxs[first]

        finalRes = res[rIdx]

        if(finalRes == 'False'):
          nbTrueFalse[0] += 1
        elif(finalRes == 'True'):
          nbTrueFalse[1] += 1

        #calcul des taux d'échantillonages
        rates[0] = nbTrueFalse[0] / (nbTrueFalse[0] + nbTrueFalse[1] if nbTrueFalse[0] + nbTrueFalse[1] > 0 else 1)
        rates[1] = nbTrueFalse[1] / (nbTrueFalse[0] + nbTrueFalse[1] if nbTrueFalse[0] + nbTrueFalse[1] > 0 else 1)

        print(rates)


      #print("algoIterations",algoIterations)
      algoIterations += 1
      broken = False
      pSelecteds = {}
      p = None
      res = []
      i = 0

      epsilon = abs(rates[0] - float(req[-2]))
      
      if epsilon <= bestEpsilon:
        bestEpsilon = epsilon
        bestSample = (rates[0],rates[1])
    
    print(algoIterations)
    print("Valeurs attendues", req[-2], req[-1])
    print("Ecart epsilon entre le résultat et le résultat attendu", epsilon)
    print("Best sample", bestSample)

      

  #obtenir le booléen selon l'index retourné par le sampling
  def getBoolAndProbaFromSample(self,p):
    i = self.sample(p)
    res = ""
    if(i == 0):
      res = "True"
    if(i == 1):
      res = "False"
    
    return res, p[i]

  #savoir quel index du tableau de proba choisir aléatoirement
  def sample(self,probas):
      rand = random.random()
      s = 0
      for i in range(len(probas)):
          s = s + float(probas[i])
          if rand < s :
              return i

class Variable:
  
  def __init__(self,name, probas = None, dependencies = None):
    if(probas == None):
      probas = []
    if(dependencies == None):
      dependencies = []
    self.name = name
    self.probas = probas
    self.dependencies = dependencies
  
  def addProbas(self, p):
    self.probas.append(p)

  def getDependencies(self):
    return self.dependencies

  def setProbas(self, probas):
    self.probas = probas
  
  def getProbas(self):
    return self.probas
  
  def getRequest(self):
    return self.request

  def toString(self):
    str = ""
    if len(self.dependencies) == 0 :
      str += "P("+self.name+") \n "+self.name
      str += "\n True \t : " + self.probas["True"]
      str += "\n False \t : " + self.probas["False"]
    else:
        str += "P("+self.name+" | " + ", ".join(self.dependencies) + " )\n"
        str += ", ".join(self.dependencies)+ '\n'
        for p in self.probas:
          str += p + '\n'
    return str

  
  #retourner la proba en fonction des valeurs données pour les dépendences
  def probaFromDependenciesVal(self,dependenciesVal):
    if(len(self.dependencies) != len(dependenciesVal)):
      return "Erreur"
    
    newProbas = []
    
    for p in self.probas:
      for i in range(len(dependenciesVal)):
        if dependenciesVal != "" and p[i] != dependenciesVal[i]:
          break
        if i == len(dependenciesVal) - 1 and (dependenciesVal == "" or p[i] == dependenciesVal[i]):
          newProbas.append(p)
    return newProbas

  #Transforme la requête en tuple pour récupérer les valeurs des dépendences
  #sous le même format que celles de la variable  
  def transformRequest(self, request):
    if len(self.dependencies) == 0:
      return tuple()
    newReq = []
    found = False

    others = request[request.find("|")+1 : request.find(")")].strip().split(",")

    '''print(others)
    print(self.dependencies)'''
    for d in self.dependencies:
      for s in filter (lambda x: d in x, others): 
        found = True
        print("found")
        newReq.append(s[s.find("=")+1:])
      if(found == False):
        newReq.append("")
      found = False
    return tuple(newReq)

  
      

class Main : 
  def __init__(self):
    pass
    
  #transcrire le fichier bif en objets Network et Variable
  def transcriptBif(self,path):
    net = None
    var = None
    probaProcessing = False
    probas = None
    ptrue = None
    pfalse = None
    p = None
    networks = []
    with open(path) as file :
      for line in file:
        if "network" in line:
          net = Network(line[line.find(" ")+1:line.rfind(" ")])
          networks.append(net)
        if "probability" in line:
          probaProcessing = True
          if "|" not in line :
            var = Variable(line[line.find("(")+2:line.find(")")-1])
          else:
            var = Variable(name = line[line.find("(")+2:line.find("|")-1],
                           dependencies = line[line.find("|")+2: line.find(")")-1].replace(" ","").split(","))
        if "table" in line :
          ptrue = line[line.find(" ",2)+1:line.find(",")]
          pfalse = line[line.rfind(" ")+1: line.find(";")]

          var.addProbas((ptrue,pfalse))

        
        if len(line) > 2 and probaProcessing and (line[line.find("(")+1] == 'F' or line[line.find("(")+1] == 'T' )  :
            if "," in line[1:line.find(")")]:
                bools = line[line.find("(")+1:line.find(")")].replace(" ","").split(",")
                ptrue = line[line.find("0"):line.rfind(",")]
                pfalse = line[line.rfind(",")+2:line.find(";")]
                var.addProbas(tuple(bools) + (ptrue,pfalse))

            else :
              p = (line[line.find("(")+1:line.find(")")], line[line.find("0"):line.find(",")],
              line[line.find(",")+2:line.find(";")])
              var.addProbas(p)
              
              
            
        if probaProcessing and "}" in line:
          probaProcessing = False
          
          '''print(var.name)
          print(var.getProbas())'''
          net.addVariable(var)
          var = None
        
        if "Prob" in line:
          net.addRequest(line)
          net = None
      return networks

  

def main():
  start_time = datetime.now()
  main = Main()
  path = "./content/bn.bif"
  networks = main.transcriptBif(path)
  '''for n in networks:
    for v in n.getVariables():
      print(v.name)'''
  firstNet = networks[1]
  firstNet.rejectSampling()
  print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

if __name__ == "__main__" :
  main()


Algorithme du likelihood sampling

In [ ]:
#likelihoodweighting
import numpy as np
def likelihoodweightingSAMPLING(evidence,variables, random_float_array):
  tabechantillon= []
  weight = 1
  j=0
  tabevnom = []
  for r in range(0,len(evidence)):
    tabevnom.append(evidence[r].name)
  r=0
  for i in range(0, len(variables)):
      if variables[i].name in tabevnom :
        tabechantillon.append([variables[i].name,evidence[r].valeur])
        
        if variables[i].probaTrue is not None:
          if evidence[r].valeur =='False':
            
            weight=weight*float(variables[i].probaFalse)
          else :
            weight=weight*float(variables[i].probaTrue)
        else :
          
          if len(variables[i].probacond)==8:
            for l in range(0,len(variables[i].probacond)):
              for m in range(0, len(variables[i].evid)):               
                  for n in range(0,i-2):               
                    if tabechantillon[n][0]==variables[i].evid[m] and tabechantillon[n][1]== variables[i].probacond[l][0] and tabechantillon[n+1][1]== variables[i].probacond[l][1] and tabechantillon[n+2][1]== variables[i].probacond[l][2] :
                      if evidence[r].valeur =='True':
                        weight=weight*float(variables[i].probacond[l][3])
                      else:
                        weight=weight*float(variables[i].probacond[l][4])
          
          elif len(variables[i].probacond)==4:
            for l in range(0,len(variables[i].probacond)):
              for m in range(0, len(variables[i].evid)):
                for n in range(0,i-1):
                  if tabechantillon[n][0]==variables[i].evid[m] and tabechantillon[n][1]== variables[i].probacond[l][0] and tabechantillon[n+1][1]== variables[i].probacond[l][1]:
                      if evidence[r].valeur =='True':
                        weight=weight*float(variables[i].probacond[l][2])
                      else:
                        weight=weight*float(variables[i].probacond[l][3])
                  
          elif len(variables[i].probacond)==2:
            for l in range(0,len(variables[i].probacond)):
              for m in range(0, len(variables[i].evid)):
                 
                for n in range(0,i):
                  if tabechantillon[n][0]==variables[i].evid[m] and tabechantillon[n][1]== variables[i].probacond[l][0]:
                      if evidence[r].valeur =='True':
                        
                        weight=weight*float(variables[i].probacond[l][1])
                      else:
                        weight=weight*float(variables[i].probacond[l][2])          
        r=r+1
      else:
        if variables[i].probaTrue is not None:

          if random_float_array[j] < float(variables[i].probaTrue) :
            tabechantillon.append([variables[i].name,'True'])
          else:
            tabechantillon.append([variables[i].name,'False']) 
          j=j+1
        else:
          
          if len(variables[i].probacond)==8:
            for l in range(0,len(variables[i].probacond)):
              for m in range(0, len(variables[i].evid)):               
                  for n in range(0,i-2):               
                    if tabechantillon[n][0]==variables[i].evid[m] and tabechantillon[n][1]== variables[i].probacond[l][0][0] and tabechantillon[n+1][1]== variables[i].probacond[l][0][1] and tabechantillon[n+2][1]== variables[i].probacond[l][0][2] :
                      if random_float_array[j] < float(variables[i].probacond[l][0][3]):
                        tabechantillon.append([variables[i].name,'True'])
                      else:
                        tabechantillon.append([variables[i].name,'False'])
                      j=j+1

          elif len(variables[i].probacond)==4:
            
            for l in range(0,len(variables[i].probacond)):
              for m in range(0, len(variables[i].evid)):
                for n in range(0,i-1):
                  if tabechantillon[n][0]==variables[i].evid[m] and tabechantillon[n][1]== variables[i].probacond[l][0][0] and tabechantillon[n+1][1]== variables[i].probacond[l][0][1]:
                    if random_float_array[j] < float(variables[i].probacond[l][0][2]):
                      tabechantillon.append([variables[i].name,'True'])
                    else:
                      tabechantillon.append([variables[i].name,'False'])
                    j=j+1
                  
          elif len(variables[i].probacond)==2:
            for l in range(0,len(variables[i].probacond)):
              for m in range(0, len(variables[i].evid)):
                for n in range(0,i):
                  if tabechantillon[n][0]==variables[i].evid[m] and tabechantillon[n][1]== variables[i].probacond[l][0][0]:
                    print(j)
                    if random_float_array[j] < float(variables[i].probacond[l][0][1]):
                      tabechantillon.append([variables[i].name,'True'])
                    else:
                      tabechantillon.append([variables[i].name,'False'])
                    j=j+1




  
  return tabechantillon,weight

class VariableRomain:
  
  def __init__(self,name, probaTrue=None, probaFalse=None,evid=None,probacond=None):
    self.name = name
    self.probaTrue = probaTrue
    self.probaFalse = probaFalse
    self.probacond= []
    self.evid=evid
  def addProbas(self, probaTrue,probaFalse):
    self.probaTrue = probaTrue
    self.probaFalse = probaFalse

  def addprobacond(self, p):
    self.probacond.append(p)

  def getDependencies(self):
    return self.dependencies

  def setProbas(self, probas):
    self.probas = probas
  
  def getProbas(self):
    return self.probas
  
  def getRequest(self):
    return self.request

class evidencevariable:

  def __init__(self,name, valeur):
    self.name = name
    self.valeur = valeur 

class reseauRomain:

  def __init__(self,name,tabvar=None,evidence=None):
    self.name=name
    self.tabvar= []

  def addVariable(self, var):
    self.tabvar.append(var)
  def addRequest(self, request):
    self.request = request
  def addevidence(self,evidence):
    self.evidence = evidence
  def getRequest(self):
    return self.request

def transcriptBif(path):
    networks = []
    probaProcessing = False
    # Ouvrir le fichier en lecture seule
    file = open(path, "r")
    # utiliser readlines pour lire toutes les lignes du fichier
    # La variable "lignes" est une liste contenant toutes les lignes du fichier
    lines = file.readlines()
    # fermez le fichier après avoir lu les lignes
    file.close()
    # Itérer sur les lignes
    for line in lines:
      if "network" in line:
          net = reseauRomain((line[line.find(" ")+1:line.rfind(" ")]))
          networks.append(net)
      if "probability" in line:
          probaProcessing = True
          if "|" not in line :
            var = VariableRomain(line[line.find("(")+2:line.find(")")-1])
          else:
            var = VariableRomain(name = line[line.find("(")+2:line.find("|")-1],
                           evid= line[line.find("|")+2: line.find(")")-1].replace(" ","").split(","))
      if "table" in line :
          ptrue = line[line.find(" ",2)+1:line.find(",")]
          pfalse = line[line.rfind(" ")+1: line.find(";")]

          var.addProbas(ptrue,pfalse)
      if len(line) > 2 and probaProcessing and (line[line.find("(")+1] == 'F' or line[line.find("(")+1] == 'T' )  :
            if "," in line[1:line.find(")")]:
                bools = line[line.find("(")+1:line.find(")")].replace(" ","").split(",")
                ptrue = line[line.find("0"):line.rfind(",")]
                pfalse = line[line.rfind(",")+2:line.find(";")]
                var.addprobacond([tuple(bools)+ (ptrue,pfalse)])

            else :
              p = [line[line.find("(")+1:line.find(")")], line[line.find("0"):line.find(",")],
              line[line.find(",")+2:line.find(";")]]
              var.addprobacond(p)
              
              
            
      if probaProcessing and "}" in line:
          probaProcessing = False
          
          '''print(var.name)
          print(var.getProbas())'''
          net.addVariable(var)
          var = None
        
      if "Prob" in line:
        
        net.addRequest(line)
        net = None
    return networks


path = "bn.bif"
networks = transcriptBif(path)



In [ ]:
#network1
MaryCallsdep=evidencevariable('MaryCalls','True')
JohnCallsdep=evidencevariable('JohnCalls','True')
evidencetab = [JohnCallsdep,MaryCallsdep]
reseau = networks[0].tabvar
a=[]
for i in range (0, 1000):
  random_float_array = np.random.uniform(0, 1,5)
  b=likelihoodweightingSAMPLING(evidencetab, reseau,random_float_array)
  a.append(b)
print(len(a))
b=0
Numerateur=0
deno=0
for b in range (0, len(a)):
  deno=deno+a[b][1]
  if a[b][0][0][1]=='False':
    Numerateur = Numerateur + a[b][1]
    Probafaux=Numerateur/deno
    Probavrai = 1- Probafaux
print("Faux = "+str(Probafaux)+",  Vrai = " +str(Probavrai))
print(networks[0].request)

In [ ]:
#Network2
N0dep=evidencevariable('N0','True')
N2dep=evidencevariable('N5','False')
evidencetab2 = [N0dep,N2dep]
reseau2 = networks[1].tabvar
a=[]
for i in range (0, 100000):
  random_float_array = np.random.uniform(0, 1,10)
  b=likelihoodweightingSAMPLING(evidencetab2, reseau2,random_float_array)
  a.append(b)
print(len(a))
print(a)
b=0
Numerateur=0
deno=0
for b in range (0, len(a)):
  deno=deno+a[b][1]
  if a[b][0][2][1]=='False':
    Numerateur = Numerateur + a[b][1]
    Probafaux=Numerateur/deno
    Probavrai = 1- Probafaux
print("Faux = "+str(Probafaux)+",  Vrai = " +str(Probavrai))
print(networks[1].request)